In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

IDIR = 'data/'

print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))
print('products {}: {}'.format(products.shape, ', '.join(products.columns)))

loading prior
loading train
loading orders
loading products
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
products (49688, 3): product_id, aisle_id, department_id


In [2]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [5]:
print("Generating product features")
prods = pd.DataFrame()
prods['product_ordered_count'] = priors.groupby(priors.product_id).size().astype(np.int32)
prods['product_reordered_count'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.int32)
prods['product_reorder_rate'] = (prods.product_reordered_count / prods.product_ordered_count).astype(np.float32)

Generating product features


In [6]:
prods.head()

,product_ordered_count,product_reordered_count,product_reorder_rate
product_id,,,
1,1852,1136,0.613391
2,90,12,0.133333
3,277,203,0.732852
4,329,147,0.446809
5,15,9,0.600000


In [7]:
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)
del prods

products.head()

,product_id,aisle_id,department_id,product_ordered_count,product_reordered_count,product_reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.613391
2,2,104,13,90.0,12.0,0.133333
3,3,94,7,277.0,203.0,0.732852
4,4,38,1,329.0,147.0,0.446809
5,5,5,13,15.0,9.0,0.600000


In [8]:
print('add order info to priors')
orders.set_index('order_id', inplace=True, drop=False)
priors = priors.join(orders, on='order_id', rsuffix='_')
priors.drop('order_id_', inplace=True, axis=1)

priors.head()

add order info to priors


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [9]:
print("Generating user features")
users = pd.DataFrame()
users['total_purchased_items'] = priors.groupby(priors.user_id).size().astype(np.int16)
users['unique_products'] = priors['product_id'].groupby(priors.user_id).apply(set)
users['unique_products_count'] = (users.unique_products.map(len)).astype(np.int16)

Generating user features


In [10]:
users.head()

,total_purchased_items,unique_products,unique_products_count
user_id,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23


In [11]:
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders['days_since_prior_order'].groupby(orders.user_id).mean().astype(np.float32)
usr['total_orders_per_user'] = orders.groupby(orders.user_id).size().astype(np.int16)

In [12]:
usr.head()

,average_days_between_orders,total_orders_per_user
user_id,,
1,19.000000,11
2,16.285715,15
3,12.000000,13
4,17.000000,6
5,11.500000,5


In [13]:
users = users.join(usr)
del usr

users['average_items_in_basket'] = (users.total_purchased_items / users.total_orders_per_user).astype(np.float32)

In [14]:
print(users.shape)
users.head()

(206209, 6)


,total_purchased_items,unique_products,unique_products_count,average_days_between_orders,total_orders_per_user,average_items_in_basket
user_id,,,,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.000000,11,5.363636
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.285715,15,13.000000
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.000000,13,6.769231
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.000000,6,3.000000
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.500000,5,7.400000


In [15]:
# Compute user x product features
priors['user_product_id'] = priors.product_id + priors.user_id * 100000

In [16]:
# Iterate dataframe rows as tuples
d = dict()
for row in priors.itertuples():
    z = row.user_product_id
    if z not in d:
        # Create new entry of 3 features
        d[z] = (1, 
                (row.order_number, row.order_id), 
                row.add_to_cart_order)
    else:
        # Update entry: 1st element: add counter, 2nd: get max/last order id
        # 3rd element: add to cart order
        d[z] = (d[z][0] + 1, 
                max(d[z][1], (row.order_number, row.order_id)),
               d[z][2] + row.add_to_cart_order)

In [17]:
# Convert to dataframe
userXproduct = pd.DataFrame.from_dict(d, orient='index')
del d
userXproduct.columns = ['nb_orders', 'last_order_id', 'sum_pos_in_cart']
userXproduct.nb_orders = userXproduct.nb_orders.astype(np.int16)
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int32)
userXproduct.sum_pos_in_cart = userXproduct.sum_pos_in_cart.astype(np.int16)
del priors

In [18]:
print(userXproduct.shape)

(13307953, 3)


In [19]:
userXproduct.head()

,nb_orders,last_order_id,sum_pos_in_cart
20227933120,5,104690,10
20227928985,5,132412,16
20227909327,1,2,3
20227945918,5,2382766,24
20227930035,3,2382766,14


In [20]:
# split to train and test orders
test_orders = orders[orders.eval_set == 'test']
train_orders = orders[orders.eval_set == 'train']

# Change index for train
train.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [21]:
print(test_orders.shape)
print(train_orders.shape)

(75000, 7)
(131209, 7)


In [22]:
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
1187899,1187899,1,train,11,4,8,14.0
1492625,1492625,2,train,15,1,11,30.0
2196797,2196797,5,train,5,0,11,6.0
525192,525192,7,train,21,2,11,6.0
880375,880375,8,train,4,1,14,10.0


In [23]:
def features(selected_orders, labels_given=False):
    print('build candidate list')
    order_list = [] # order_id
    product_list = [] # product_id
    labels = [] # True/False if found in train.index
    i=0
    for row in selected_orders.itertuples():
        # Increase counter
        i+=1
        if i%10000 == 0: print('order row',i)
        order_id = row.order_id
        user_id = row.user_id
        # Query unique products from users table for each user_id
        user_products = users.unique_products[user_id]
        # And append them to product_list
        product_list += user_products
        # Append order_id tied to these unique products
        order_list += [order_id] * len(user_products)
        if labels_given:
            labels += [(order_id, product) in train.index for product in user_products]
            
    # create df
    df = pd.DataFrame({'order_id': order_list, 'product_id': product_list}, dtype=np.int32)
    labels = np.array(labels, dtype=np.int8)
    del order_list
    del product_list
        
    print('user related features')
    df['user_id'] = df.order_id.map(orders.user_id)
    df['user_total_orders'] = df.user_id.map(users.total_orders_per_user)
    df['user_total_items'] = df.user_id.map(users.total_purchased_items)
    df['total_distinct_items'] = df.user_id.map(users.unique_products_count)
    df['user_average_days_between_orders'] = df.user_id.map(users.average_days_between_orders)
    df['user_average_basket'] =  df.user_id.map(users.average_items_in_basket)
        
    print('order related features')
    # df['dow'] = df.order_id.map(orders.order_dow)
    df['order_hour_of_day'] = df.order_id.map(orders.order_hour_of_day)
    df['days_since_prior_order'] = df.order_id.map(orders.days_since_prior_order)
    df['days_since_ratio'] = df.days_since_prior_order / df.user_average_days_between_orders
    
    print('product related features')
    df['aisle_id'] = df.product_id.map(products.aisle_id)
    df['department_id'] = df.product_id.map(products.department_id)
    df['product_orders'] = df.product_id.map(products.product_ordered_count).astype(np.int32)
    df['product_reorders'] = df.product_id.map(products.product_reordered_count)
    df['product_reorder_rate'] = df.product_id.map(products.product_reorder_rate)
    
    print('user_X_product related features')
    df['z'] = df.user_id * 100000 + df.product_id
    df.drop(['user_id'], axis=1, inplace=True)
    df['UP_orders'] = df.z.map(userXproduct.nb_orders)
    df['UP_orders_ratio'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
    df['UP_last_order_id'] = df.z.map(userXproduct.last_order_id)
    df['UP_average_pos_in_cart'] = (df.z.map(userXproduct.sum_pos_in_cart) / df.UP_orders).astype(np.float32)
    df['UP_reorder_rate'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
    df['UP_orders_since_last'] = df.user_total_orders - df.UP_last_order_id.map(orders.order_number)
    df['UP_delta_hour_vs_last'] = abs(df.order_hour_of_day - df.UP_last_order_id.map(orders.order_hour_of_day)).map(lambda x: min(x, 24-x)).astype(np.int8)
    #df['UP_same_dow_as_last_order'] = df.UP_last_order_id.map(orders.order_dow) == \
    #                                              df.order_id.map(orders.order_dow)
    
    df.drop(['UP_last_order_id', 'z'], axis=1, inplace=True)
    print(df.dtypes)
    print(df.memory_usage())
    
    return (df, labels)

df_train, labels = features(train_orders, labels_given=True)

build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
order row 80000
order row 90000
order row 100000
order row 110000
order row 120000
order row 130000
user related features
order related features
product related features
user_X_product related features
order_id                              int32
product_id                            int32
user_total_orders                     int16
user_total_items                      int16
total_distinct_items                  int16
user_average_days_between_orders    float32
user_average_basket                 float32
order_hour_of_day                      int8
days_since_prior_order              float32
days_since_ratio                    float32
aisle_id                              uint8
department_id                         uint8
product_orders                        int32
product_reorders                    float64
product_reorder_rate                float32
UP_or

In [24]:
df_train.head()

,order_id,product_id,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_hour_of_day,days_since_prior_order,days_since_ratio,...,department_id,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_reorder_rate,UP_orders_since_last,UP_delta_hour_vs_last
0,1187899,17122,11,59,18,19.0,5.363636,8,14.0,0.736842,...,4,13880,9377.0,0.675576,1,0.090909,6.0,0.090909,6,7
1,1187899,196,11,59,18,19.0,5.363636,8,14.0,0.736842,...,7,35791,27791.0,0.776480,10,0.909091,1.4,0.909091,1,0
2,1187899,26405,11,59,18,19.0,5.363636,8,14.0,0.736842,...,17,1214,536.0,0.441516,2,0.181818,5.0,0.181818,7,1
3,1187899,46149,11,59,18,19.0,5.363636,8,14.0,0.736842,...,7,8558,6953.0,0.812456,3,0.272727,3.0,0.272727,1,0
4,1187899,14084,11,59,18,19.0,5.363636,8,14.0,0.736842,...,16,15935,12923.0,0.810982,1,0.090909,2.0,0.090909,10,0


In [26]:
# Build candidate list for test dataset
df_test, _ = features(test_orders)

build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
user related features
order related features
product related features
user_X_product related features
order_id                              int32
product_id                            int32
user_total_orders                     int16
user_total_items                      int16
total_distinct_items                  int16
user_average_days_between_orders    float32
user_average_basket                 float32
order_hour_of_day                      int8
days_since_prior_order              float32
days_since_ratio                    float32
aisle_id                              uint8
department_id                         uint8
product_orders                        int32
product_reorders                    float64
product_reorder_rate                float32
UP_orders                             int16
UP_orders_ratio                     float32
UP_average_pos_in

In [28]:
import pickle

with open('data.pkl','wb') as f:
    pickle.dump([df_train, labels, df_test, test_orders],f)

print("File saved")

File saved
